In [2]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pickle

def pipeline_model_from_camera(scale_factor=1.5):
    # Đường dẫn đến các mô hình phát hiện và nhận diện khuôn mặt
    faceDetectionModel = '/thigiacmaytinh/models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
    faceDetectionProto = '/thigiacmaytinh/models/deploy.proto.txt'
    faceDescriptor = '/thigiacmaytinh/models/openface.nn4.small2.v1.t7'

    # Sử dụng OpenCV DNN để đọc mô hình
    detectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)
    descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptor)
    
    # Load mô hình nhận diện khuôn mặt
    face_recognition_model = pickle.load(open('/thigiacmaytinh/ml_face_person_identity.pkl', mode='rb'))
    
    # Mở camera
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise Exception("Không thể mở camera")

    while True:
        # Đọc khung hình từ camera
        ret, img = cap.read()
        if not ret:
            break

        # Phóng to khung hình bằng scale_factor
        img = cv2.resize(img, (0, 0), fx=scale_factor, fy=scale_factor)

        image = img.copy()
        h, w = img.shape[:2]

        # Face detection: Tạo blob từ hình ảnh
        img_blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB=False, crop=False)
        detectorModel.setInput(img_blob)
        detections = detectorModel.forward()

        # Kiểm tra và xử lý các phát hiện khuôn mặt
        if len(detections) > 0:
            for i, confidence in enumerate(detections[0, 0, :, 2]):
                # Ngưỡng tin cậy cho nhận diện khuôn mặt
                if confidence > 0.8:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    startx, starty, endx, endy = box.astype(int)

                    # Vẽ hình chữ nhật quanh khuôn mặt
                    cv2.rectangle(image, (startx, starty), (endx, endy), (0, 255, 0), 2)

                    # Trích xuất ROI của khuôn mặt
                    face_roi = img[starty:endy, startx:endx]
                    if face_roi.size == 0:
                        continue

                    # Feature extraction từ khuôn mặt
                    face_blob = cv2.dnn.blobFromImage(face_roi, 1/255, (96, 96), (0, 0, 0), swapRB=True, crop=True)
                    descriptorModel.setInput(face_blob)
                    vectors = descriptorModel.forward()

                    # Dự đoán với mô hình máy học
                    face_name = face_recognition_model.predict(vectors)[0]
                    face_score = 1.0  # Có thể xử lý xác suất nếu cần

                    # Văn bản hiển thị tên và xác suất
                    text_face = '{} : {:.0f} %'.format(face_name, 100 * face_score)
                    
                    # Thêm hiệu ứng bóng cho văn bản
                    cv2.putText(image, text_face, (startx, starty - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                    cv2.putText(image, text_face, (startx, starty - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Hiển thị hình ảnh đã phóng to với khuôn mặt được nhận diện
        cv2.imshow('Detected Faces', image)

        # Thoát khi nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Giải phóng camera và đóng cửa sổ
    cap.release()
    cv2.destroyAllWindows()

# Gọi hàm để chạy nhận diện từ camera và phóng to video lên 1.5 lần
pipeline_model_from_camera(scale_factor=1)
